<a href="https://colab.research.google.com/github/Seyoung-Jung/Tobigs-HW/blob/master/week4/SVM/assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### TOBIG'S 14기 정규세션 4주차 SVM 
### ASSIGNMENT2
---

 Anomaly detection 데이터셋을 SVM을 이용해서 판별하는 과제로 여러가지 방법을 사용해서 트레이닝 해보고 결과를 비교해주면됩니다!

 데이터 분포 : True : 0.17프로 나머지 다 False, 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
# Anomaly detection(사기감지 데이터) 로드
data = pd.read_csv('./drive/My Drive/Tobigs/week4_SVM/anomaly-detection/creditcard.csv')
print(data.columns)

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


In [ ]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
X = data.iloc[:,:30]
y = data.iloc[:,-1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [ ]:
scal = StandardScaler()
X_train = scal.fit_transform(X_train)
X_test = scal.transform(X_test)

split한 후에 scaling 해주어야 한다. X_train에 fit하여 transform한 후 이를 X_test에 적용시켜 마무리해준다.

In [ ]:
parameters = [{'C': [0.1, 1, 10], 'kernel': ['linear']},
              {'C': [0.1, 1, 10], 'kernel': ['rbf'], 'gamma': [0.01, 0.1, 1]}]

In [ ]:
# from sklearn.model_selection import GridSearchCV

# svm= SVC()
# svm_grid = GridSearchCV(svm, parameters, cv=3, scoring='accuracy', verbose = 3)
# svm_grid.fit(X_train, y_train)
# best_accuracy = svm_grid.best_score_
# print("best accuracy : ", best_accuracy)

# best_parameters = svm_grid.best_params_
# print("best parameters : ", best_parameters)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] C=0.1, kernel=linear ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ C=0.1, kernel=linear, score=0.999, total= 4.3min
[CV] C=0.1, kernel=linear ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.3min remaining:    0.0s


[CV] ................ C=0.1, kernel=linear, score=0.999, total=  21.8s
[CV] C=0.1, kernel=linear ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.7min remaining:    0.0s


[CV] ................ C=0.1, kernel=linear, score=0.999, total=  24.7s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.999, total= 8.7min
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.999, total= 2.9min
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.999, total= 2.7min
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.999, total=34.7min
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.999, total=26.1min
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.999, total=77.8min
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .

전체 데이터를 가지고 gridSearch를 두 번이나 시도해봤는데 계속 노트북 CPU 연결이 끊겨서,,, 우선 데이터가 굉장히 크므로  앤드류응 교수님의 말씀에 따라 커널 없는  SVM 모델을 만들기로 했습니다!

In [ ]:
from sklearn import metrics
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print("Accuracy score:",metrics.accuracy_score(y_test, y_pred))
metrics.confusion_matrix(y_test, y_pred)

Accuracy score: 0.999403110845827


array([[56839,    14],
       [   20,    89]])

정확도는 0.999로 높게 나왔지만 confusion matrix를 살펴보면, 데이터의 imbalance로 인해 거의 사기가 아니라고만 예측하고 있음을 알 수 있다.
이를 해결하기 위해 under sampling을 진행한 후 다시 학습을 진행한다.

under sampling 이란? : 데이터 불균형에서 소수 파트에 맞춰 적지만 balnace하게 샘플링하는 방법

In [ ]:
len_frauds = len(data[data["Class"]==1])  # 사기인 데이터 길이

fraud_index = np.array(data[data["Class"]==1].index)  # 사기인 데이터 인덱스
normal_index = data[data["Class"]==0].index  # 사기가 아닌 데이터 인덱스

random_normal_index = np.random.choice(normal_index, len_frauds, replace= False)  # 사기인 데이터와 같은 길이만큼 사기가 아닌 데이터 인덱스 추출
random_normal_index = np.array(random_normal_index)
undersampled_index = np.concatenate([fraud_index, random_normal_index])  # 동일한 길이의 사기 데이터 인덱스와 사기 아닌 데이터 인덱스
undersampled_data = data.iloc[undersampled_index, :]  # balance한 undersampled data 완성

print('fraud:', len(undersampled_data[undersampled_data['Class']==1]))
print('normal:', len(undersampled_data[undersampled_data['Class']==0]))

fraud: 492
normal: 492


In [ ]:
X2 = undersampled_data.iloc[:,:30]
y2 = undersampled_data.iloc[:,-1]
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=48)

In [ ]:
scal = StandardScaler()
X2_train = scal.fit_transform(X2_train)
X2_test = scal.transform(X2_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

svm= SVC()
svm_grid = GridSearchCV(svm, parameters, cv=10, scoring='accuracy', verbose = 3)
svm_grid.fit(X2_train, y2_train)

best_parameters = svm_grid.best_params_
print("best parameters : ", best_parameters)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.937, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.911, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.924, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.937, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.911, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.924, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] .........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................. C=1, kernel=linear, score=0.974, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.936, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.949, total=   0.0s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.924, total=   0.1s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.886, total=   0.0s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.924, total=   0.0s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.937, total=   0.1s
[CV] C=10, kernel=linear .............................................
[CV] .

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    3.2s finished


under sampling한 데이터로부터 gridSearch로 얻은 best parameters를 가지고 모델 학습을 진행한다.

In [ ]:
svm_best_parameters = SVC(C=10, gamma=0.01, kernel='rbf')
svm_best_parameters.fit(X2_train, y2_train)
y2_pred = svm_best_parameters.predict(X2_test)
print("Accuracy score:",metrics.accuracy_score(y2_test, y2_pred))
metrics.confusion_matrix(y2_test, y2_pred)

Accuracy score: 0.934010152284264


array([[ 83,   3],
       [ 10, 101]])

balance한 결과를 볼 수 있다.

data split을 다른 비율로 두고 돌려보면 어떨까? 이렇게 imbalance한 데이터에서 꼭 train에 80%를 줄 필요는 없다고 느껴진다. test 비율을 80%로 해보기로 한다. (지난 기수 우수 코드를 참고했습니다.)

In [ ]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X, y, test_size = 0.80, random_state=0)

In [ ]:
scaler = StandardScaler()
X3_train = scaler.fit_transform(X3_train)
X3_test = scaler.transform(X3_test)

In [ ]:
svm= SVC()
svm_grid_2 = GridSearchCV(svm, parameters, cv=10, scoring='accuracy', verbose = 3)
svm_grid_2.fit(X3_train, y3_train)

best_parameters_2 = svm_grid_2.best_params_
print("best parameters : ", best_parameters_2)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV] C=0.1, kernel=linear ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ C=0.1, kernel=linear, score=0.999, total=   0.9s
[CV] C=0.1, kernel=linear ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV] ................ C=0.1, kernel=linear, score=0.999, total=   0.9s
[CV] C=0.1, kernel=linear ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.8s remaining:    0.0s


[CV] ................ C=0.1, kernel=linear, score=0.999, total=   1.6s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.999, total=   1.1s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.999, total=   1.2s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=1.000, total=   1.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.999, total=   0.7s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.999, total=   0.8s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.999, total=   0.8s
[CV] C=0.1, kernel=linear ............................................
[CV] .

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 153.8min finished


best parameters :  {'C': 0.1, 'kernel': 'linear'}


In [ ]:
svm_best_parameters_2 = SVC(C=0.1, kernel='linear')
svm_best_parameters_2.fit(X3_train, y3_train)
y3_pred = svm_best_parameters_2.predict(X3_test)
print("Accuracy score:",metrics.accuracy_score(y3_test, y3_pred))
metrics.confusion_matrix(y3_test, y3_pred)

Accuracy score: 0.9993723803462881


array([[227392,     63],
       [    80,    310]])

train 비율을 줄였음에도 불구하고 정확도는 비슷하게 매우 높게 나왔다. 데이터가 imbalance한 경우에는 train을 많이 하지 않아도 동일한 성능을 보임을 알 수 있다.